In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [3]:
data=sio.loadmat('PA_1_matlab/count_data.mat')

In [4]:
print data

{'ym': array([[16.0525]]), 'testy': array([[ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 15.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 14.9475],
       [ 14.9475],
       [ 13.9475],
       [ 13.9475],
       [ 13.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 14.9475],
       [ 13.9475],
       [ 12.9475],
       [ 12.9475],
       [ 12.9475],
       [ 12.9475],
       [ 12.94

In [5]:
testx=data['testx']
testy=data['testy']
trainy=data['trainy']
trainx=data['trainx']
ym=data['ym']

In [6]:
print testx.shape
print testy.shape
print trainx.shape
print trainy.shape
print ym.shape

(9, 600)
(600, 1)
(9, 400)
(400, 1)
(1, 1)


In [7]:
print len(testy)

600


In [9]:
#φ(x) = [x1,x2,x3,x4...x9,]
#so Phi=trainx
def phi(trainx):
    d,m=trainx.shape
    phi=np.zeros((2*d,m))
    temp2=np.zeros((d,m))
    temp1=np.zeros((d,m))
    for i in range(d):
        for j in range(m):
            temp2[i][j]=np.power(trainx[i][j],2)
            temp1[i][j]=trainx[i][j]
    phi=np.vstack((temp1,temp2))
    return phi

def meanabsoluteerror(testy,f_l):
    sum=0
    m=len(testy)
    for i in range(m):
        error=np.absolute(f_l[i][0]-testy[i][0])
        sum+=error
    mae=sum/m
    return mae

def meansquareerror(polyy,f_l):
    sum=0
    m=len(polyy)
    for i in range(m):
        error=f_l[i][0]-polyy[i][0]
        sum+=np.power(error,2)
    mse=sum/m
    return mse

def paraestimate(Phi,sampy):
    A=np.linalg.inv(np.dot(Phi,Phi.T))
    theta_ls=np.dot(np.dot(A,Phi),sampy)
    return theta_ls
Phi=phi(trainx)
theta_real=paraestimate(Phi,trainy)
print theta_real.shape
f_ls=np.dot(Phi.T,theta_real)
print f_ls.shape
print meanabsoluteerror(testy.T,f_ls)
print meansquareerror(testy.T,f_ls)

(18, 1)
(400, 1)
24.357632762377484
593.294273786845


In [28]:
##LR
def paraestimateReg(Phi,sampy,lmb=1):
    A=np.linalg.inv(np.dot(Phi,Phi.T)+lmb*np.eye(9))
    theta_ls=np.dot(np.dot(A,Phi),sampy)
    return theta_ls
theta_ls=paraestimateReg(trainx,trainy)
print theta_ls.shape
f_lr=np.dot(testx.T,theta_ls)
print f_lr.shape
print meanabsoluteerror(testy,f_lr)
print meansquareerror(testy,f_lr)

(9, 1)
(600, 1)
1.282432855732885
2.6187339222418937


In [30]:
#LASSO
import scipy.optimize as opt

def paraestimateLasso(Phi,sampy,lmb=1):
    H=np.hstack((np.vstack((np.dot(Phi,Phi.T),np.dot(-Phi,Phi.T))),np.vstack((np.dot(-Phi,Phi.T),np.dot(Phi,Phi.T)))))
    fa=np.vstack((np.dot(Phi,sampy),-np.dot(Phi,sampy)))
    f=lmb*np.ones((18,1))-fa
    x_ans=np.zeros((18,1))
    fun=lambda x: float(0.5*np.dot((np.dot(x.T,H)),x)+np.dot(f.T,x))
    cons=({'type': 'ineq', 'fun': lambda x: x})
    x_ans_true=opt.minimize(fun,x_ans,constraints=cons,method="SLSQP")# these 3 lines use Opt way to get the min of function
    
    theta_plus=x_ans_true.x[0:9]
    theta_sub=x_ans_true.x[9:18]
    theta=theta_plus-theta_sub
    theta=np.expand_dims(theta, axis=1)
    return theta

theta_lasso=paraestimateLasso(trainx,trainy,lmb=1)
f_lasso=np.dot(testx.T,theta_lasso)
print f_lasso.shape
print meanabsoluteerror(testy,f_lasso)
print meansquareerror(testy,f_lasso)

(600, 1)
1.3008913835130993
2.7828235164999673


In [31]:
def ArrayToList(array):
    list=[]
    for i in range(len(array)):
        list.append(float(array[i]))
    return list

def paraestimateRobustreg(Phi,sampy,lmb=1):
    m=len(sampy)
    k=len(Phi)
    temp1=np.row_stack((-Phi.T,Phi.T))
    temp2=np.row_stack((-np.eye(m),-np.eye(m)))
    A=np.column_stack((temp1,temp2))
    b=np.row_stack((-sampy,sampy))
    f=np.row_stack((np.zeros((k,1)),np.ones((m,1))))
    X=np.zeros((m+k, 1))
    B=ArrayToList(b)
    fun=lambda x:np.dot(f.T,x)
    cons=({'type': 'ineq', 'fun': lambda x:B-np.dot(A, x)})#B should be list then can get constraint
    x_ans=opt.minimize(fun,X,constraints=cons, method="SLSQP")
    
    theta=x_ans.x[0:k]
    theta=np.expand_dims(theta, axis=1)
    return theta
theta_rr=paraestimateRobustreg(trainx,trainy,lmb=1)
f_rr=np.dot(testx.T,theta_rr)
print f_rr.shape
print meanabsoluteerror(testy,f_rr)
print meansquareerror(testy,f_rr)

(600, 1)
1.3645672474557409
3.1189980526815244


In [33]:
def esitmatesigma(Phi,alpha=1,sigma_square=5):
    temp=(1/float(sigma_square))*np.dot(Phi,Phi.T)
    m=len(Phi)
    SIGMA=np.linalg.inv(temp+(1/float(alpha))*np.eye(m))
    return SIGMA

def estimatemju(sampy,Phi,SIGMA,sigma_square=5):
    mju=(1/float(sigma_square))*np.dot(np.dot(SIGMA,Phi),sampy)
    return mju

def gaussianpredictedsigma(SIGMA,polyx_l):
    polyphi=np.zeros((1,6))
    for i in range(6):
        polyphi[0][i]=np.power(polyx_l,i)
    res=np.dot(np.dot(polyphi,SIGMA),polyphi.T)
    return res

SIGMA=esitmatesigma(trainx,alpha=1,sigma_square=5)
mju=estimatemju(trainy,trainx,SIGMA,sigma_square=5)
f_br=np.dot(testx.T,mju)
print meanabsoluteerror(testy,f_br)
print meansquareerror(testy,f_br)


1.4486049524761022
3.1462528214455157


In [52]:
temp1=np.zeros((len(trainx),len(trainx[0])))
for j in range(len(trainx[0])):
    for i in range(len(trainx)):
        temp1[i][j]=np.power(trainx[i][j],2)
phi=np.row_stack((trainx,temp1))

temp2=np.zeros((len(testx),len(testx[0])))
for j in range(len(testx[0])):
    for i in range(len(testx)):
        temp2[i][j]=np.power(testx[i][j],2)
test_phi=np.row_stack((testx,temp2))

phi=np.row_stack((trainx,temp1))
print phi[11][0]
print trainx[2][0]

0.6287867148877597
-0.7929607272039138


In [53]:
#LS
theta_real=paraestimate(phi,trainy)
print theta_real.shape
f_ls=np.dot(test_phi.T,theta_real)
print f_ls.shape
print meanabsoluteerror(testy,f_ls)
print meansquareerror(testy,f_ls)

(18, 1)
(600, 1)
1.3267461241105298
2.9235940267301013


In [54]:
def paraestimateReg(Phi,sampy,lmb=1):
    A=np.linalg.inv(np.dot(Phi,Phi.T)+lmb*np.eye(18))
    theta_ls=np.dot(np.dot(A,Phi),sampy)
    return theta_ls
theta_ls=paraestimateReg(phi,trainy)
print theta_ls.shape
f_lr=np.dot(test_phi.T,theta_ls)
print f_lr.shape
print meanabsoluteerror(testy,f_lr)
print meansquareerror(testy,f_lr)

(18, 1)
(600, 1)
1.213698757046865
2.467773902567654


In [55]:
#LASSO
import scipy.optimize as opt

def paraestimateLasso(Phi,sampy,lmb=1):
    H=np.hstack((np.vstack((np.dot(Phi,Phi.T),np.dot(-Phi,Phi.T))),np.vstack((np.dot(-Phi,Phi.T),np.dot(Phi,Phi.T)))))
    fa=np.vstack((np.dot(Phi,sampy),-np.dot(Phi,sampy)))
    f=lmb*np.ones((36,1))-fa
    x_ans=np.zeros((36,1))
    fun=lambda x: float(0.5*np.dot((np.dot(x.T,H)),x)+np.dot(f.T,x))
    cons=({'type': 'ineq', 'fun': lambda x: x})
    x_ans_true=opt.minimize(fun,x_ans,constraints=cons,method="SLSQP")# these 3 lines use Opt way to get the min of function
    
    theta_plus=x_ans_true.x[0:18]
    theta_sub=x_ans_true.x[18:36]
    theta=theta_plus-theta_sub
    theta=np.expand_dims(theta, axis=1)
    return theta

theta_lasso=paraestimateLasso(phi,trainy,lmb=1)
f_lasso=np.dot(test_phi.T,theta_lasso)
print f_lasso.shape
print meanabsoluteerror(testy,f_lasso)
print meansquareerror(testy,f_lasso)

(600, 1)
1.2425737603631541
2.545271416551002


In [56]:

theta_rr=paraestimateRobustreg(phi,trainy,lmb=1)
f_rr=np.dot(test_phi.T,theta_rr)
print f_rr.shape
print meanabsoluteerror(testy,f_rr)
print meansquareerror(testy,f_rr)

(600, 1)
1.307928627386951
2.898128040979179


In [57]:
def esitmatesigma(Phi,alpha=1,sigma_square=5):
    temp=(1/float(sigma_square))*np.dot(Phi,Phi.T)
    m=len(Phi)
    SIGMA=np.linalg.inv(temp+(1/float(alpha))*np.eye(m))
    return SIGMA

def estimatemju(sampy,Phi,SIGMA,sigma_square=5):
    mju=(1/float(sigma_square))*np.dot(np.dot(SIGMA,Phi),sampy)
    return mju

def gaussianpredictedsigma(SIGMA,polyx_l):
    polyphi=np.zeros((1,6))
    for i in range(6):
        polyphi[0][i]=np.power(polyx_l,i)
    res=np.dot(np.dot(polyphi,SIGMA),polyphi.T)
    return res

SIGMA=esitmatesigma(phi,alpha=1,sigma_square=5)
mju=estimatemju(trainy,phi,SIGMA,sigma_square=5)
f_br=np.dot(test_phi.T,mju)
print meanabsoluteerror(testy,f_br)
print meansquareerror(testy,f_br)

1.2968993099372872
2.913107415872251
